In [1]:
from Project.Database import Db
from optimisation_problem import hourly_house_df, slice_emission_vector, NZERTF_optimiser

In [2]:
def find_emissions(df, emission_vec):
    energy_vec = df.loc[lambda self: self['Day'] == day, 'Consumption'].reset_index(drop=True)
    energy_consumed = energy_vec.multiply(emission_vec)
    for index, emission in energy_consumed[lambda self: self > 0].iteritems():
        df.loc[lambda self: (self['Day'] == day) & (self['Hour'] == index), 'Emission'] += emission
    return df

In [ ]:
# year2_orig = year2.copy()
# 
# for app in movable_appliances:
#     lifespan = app_stats[app]['Timespan'][0]
#     hour_slots = int(np.ceil(lifespan / 60))
#     average_power_consumption = average_power_consumption_df.loc[lambda self: self.index == app].values[0]
#     energy_consumption_per_hour = round(average_power_consumption * lifespan / 60_000 / hour_slots, 4)
# 
#     for day in year2_orig['Day'].unique():
#         emission_vector = slice_emission_vector(day)
#         energy_vector = year2_orig.loc[lambda self: self['Day'] == day, app].multiply(
#             energy_consumption_per_hour).reset_index(drop=True)
#         energy_consumed = energy_vector.multiply(emission_vector)
#         for index, emission in energy_consumed[lambda self: self > 0].iteritems():
#             year2_orig.loc[lambda self: (self['Day'] == day) & (self['Hour'] == index), 'Emission'] += emission
# 
# year2_orig

In [16]:
meta = Db.load_data(meta=True, hourly=False, year=2, consumption=False).loc[
    lambda self: (~self['Consumer_Match'].isna()), 'Consumer_Match']

production = Db.load_data(
    consumption=False,
    production=True,
    year=2)

production = production.groupby(
    [production.index.strftime('%Y-%m-%d'),
     production.index.hour]).sum()['CO2(Grams)/kWh']

movable_appliances = ['Load_StatusApplianceDishwasher', 'Load_StatusPlugLoadVacuum', 'Load_StatusClothesWasher', 'Load_StatusDryerPowerTotal', 'Load_StatusPlugLoadIron']

appliances = ['Timestamp'] + meta.tolist()

# with redundancy <- w.r
# without redundancy <- w.o.r
# movable appliances unoptimised <- m.a.u.o
# movable appliances optimised <- m.a.o
NZERTF_optimisation = {
    'w.r': Db.load_data(year=2, hourly=False)[appliances],
    'w.o.r': Db.load_data(year=2, hourly=False, with_redundancy=False)[appliances]
}

NZERTF_optimisation.update({
    'm.a.u.o': NZERTF_optimisation['without_redundancy'].copy()[['Timestamp'] + movable_appliances]
})

NZERTF_emission = {}

for key in NZERTF_optimisation.keys():
    NZERTF_optimisation[key] = hourly_house_df(NZERTF_optimisation[key], aggregate_func='sum')
    NZERTF_optimisation[key]['Consumption'] = NZERTF_optimisation[key][meta.tolist()].sum(1).div(60_000)
    try:
        NZERTF_optimisation[key].drop(meta.tolist(), inplace=True, axis=1)
    except:
        NZERTF_optimisation[key].drop(meta.loc[movable_appliances].tolist(), inplace=True, axis=1)
    
    NZERTF_emission.update({
        key: NZERTF_optimisation[key].groupby(['Day']).sum()['Emission'].sum()
    })
    for day in NZERTF_optimisation[key]['Day'].unique():
        emission_vector = slice_emission_vector(production_vectors=production,
                                                day_number=day)
        NZERTF_optimisation[key] = find_emissions(NZERTF_optimisation[key],
                                                emission_vec=emission_vector)

NZERTF_optimisation.update({
    'm.a.o': NZERTF_optimiser()
})

NZERTF_optimisation

      Day  Hour     Emission  Consumption
0       0     0  1305.908727     1.778433
1       0     1  1577.101483     2.096081
2       0     2  1357.877709     1.842206
3       0     3  1338.298088     1.825044
4       0     4  1347.814095     1.843692
...   ...   ...          ...          ...
8667  364    19  6204.609403     9.064332
8668  364    20  5643.307791     8.095270
8669  364    21  4622.137879     8.481025
8670  364    22  4098.899884     9.746986
8671  364    23  1944.977733     4.625063

[8672 rows x 4 columns]
      Day  Hour     Emission  Consumption
0       0     0  1305.908727     1.778433
1       0     1  1577.101483     2.096081
2       0     2  1357.877709     1.842206
3       0     3  1338.298088     1.825044
4       0     4  1347.814095     1.843692
...   ...   ...          ...          ...
8667  364    19  5280.096184     7.713708
8668  364    20  5218.675840     7.486140
8669  364    21  4622.137879     8.481025
8670  364    22  4060.237849     9.655049
8671  364

In [ ]:
redundancy_reduction = NZERTF_optimisation['']['Emission'].sum() - NZERTF_optimisation['']['Emission'].sum()
optimisation_reduction = NZERTF_optimisation['']['Emission'].sum() - NZERTF_optimisation['']['Emission'].sum()
total_reduction = redundancy_reduction + optimisation_reduction

In [ ]:
redundancy_reduction

In [ ]:
optimisation_reduction

In [ ]:
total_reduction